### 1. 数据基本处理
- 先把数据和标签读取进来，再进行基本的删除工作，有：删除丢失数据，删除错误数据

In [72]:
import numpy as np
from util import *
from PIL import Image

In [8]:
# 删除丢失数据
# 得到标签列表和图片数据
label_Path = '../face/'
img_Path = '../face/rawdata/'
label_list = get_label_list(label_Path)
# 得到图片数据，对于标签缺失的数据直接删除
# get_img_data(img_Path, label_list)

# 得到删除后标签的数据，对于标签缺失的数据直接删除
labe_dict = []
for _dict in label_list :           
    if _dict['missing'] != 'true' :
        labe_dict.append(_dict)
label_list = labe_dict

# 加载图片数据，并输出此时 数据和标签 的大小
img_save_Path = './ImgData/img_old.npy'
img_list = np.load(img_save_Path) # (3993*(128*128))
print(img_list.shape)
print(np.array(labe_dict).shape)

(3993, 16384)
(3993,)


In [9]:
# 删除错误数据
img_list, label_list = DeleteBadData(img_list, label_list)
print(img_list.shape)
print(np.array(label_list).shape)

数据错误图片对应的id为： ['3283', '3860', '3861', '3862', '3883', '4125', '4146', '4237', '4267', '4295', '4335', '4354', '4429', '4452', '4498', '4566', '4637', '4679', '4710', '4779', '4908', '4992', '5076', '5113', '2099', '2100', '2101', '2102', '2103', '2104', '2105', '2106']
(3961, 16384)
(3961,)


### 2. 保存处理好的图像数据，原图像数据和上采样后的图像数据

因为我们使用resnet50作为迁移学习模型，其要求图像为（224，224），故需要把（128，128）的图像进行上采样放大为原来的两倍，变为（256，256），方便使用resnet50模型

In [20]:
# 按性别名称保存jpg图像到face_data/sex_origina文件夹中
# 保存的图名为 '标签.序号.jpg' ，如 'female.00001.jpg'
# 图片大小为（128，128）
save_Path = '../face_data/sex_original/img/'
id_to_indx = []
indx = 1
for i in range(img_list.shape[0]) :
    sex = label_list[i]['sex']
    img = img_list[i]
    img = img.reshape(128,128)
    img = Image.fromarray(img)
    s=".%05d"%indx
    img.save(save_Path + sex + s + '.jpg')
    id = label_list[i]['id']
    id_to_indx.append(id)
    indx += 1
np.save( '../face_data/sex_original/id_to_indx.npy', np.array(id_to_indx))
print('success')

success


In [16]:
img_list.shape[0]

3961

- 把这3961个图像上采样变为（256，256）
- 采用双线性插值（bilinear）

In [26]:
import torch
import torch.nn as nn

In [53]:
# 把上采样后的jpg图像到face_data/sex_upsample文件夹中
# 保存的图名为 '标签.序号.jpg' ，如 'female.00001.jpg'
# 图片大小为（256，256）
def img_upsample(img_list, label_list, save_Path='../face_data/sex_upsample/'):
    id_to_indx = []
    indx = 1
    Bilinear2d = nn.UpsamplingBilinear2d(scale_factor=2) # 双线性插值类方法
    for i in range(img_list.shape[0]) :
        sex = label_list[i]['sex']
        img = img_list[i]
        img = img.reshape(128,128)
        img = torch.from_numpy(img).view(1, 1, 128, 128) # 转换为tensor
        img = Bilinear2d(img.float())        # 上采样只能在tensor浮点数运行
        img = img.numpy().reshape(256,256)   # 转换为numpy
        img = Image.fromarray(np.uint8(img)) # 图片要使用np.uint8
        s=".%05d"%indx
        img.save(save_Path + 'img/' + sex + s + '.jpg')
        id = label_list[i]['id']
        id_to_indx.append(id)
        indx += 1
    np.save( save_Path + 'id_to_indx.npy', np.array(id_to_indx))
    print('success')
    
img_upsample(img_list, label_list)

success


- 对比展示

In [57]:
# 取img_list第一幅图像进行展示
img1 = img_list[0].reshape(128,128)
img1 = torch.from_numpy(img1).view(1, 1, 128, 128)

# 上采样类
Bilinear2d = nn.UpsamplingBilinear2d(scale_factor=2)
lingNearest2d = nn.UpsamplingNearest2d(scale_factor=2)

img_b = Bilinear2d(img1.float())
img_l = lingNearest2d(img1.float())

# 将数组转为图像的过程中，数组中的数据是浮点型（F）无法转为图像，
# 因此需要改变image的类型
img_b = img_b.numpy().reshape(256,256)   # 转换为numpy
img_l = img_l.numpy().reshape(256,256) 

img_b_RGB = Image.fromarray(img_b).convert('RGB') # 转换为RGB
img_b = Image.fromarray(np.uint8(img_b)) # 灰度图片要使用np.uint8
img_l = Image.fromarray(np.uint8(img_l)) 

im = img_list[0].reshape(128,128)  # 原图
im = Image.fromarray(im)

# 保存图片
root_path = '../face_data/sex_上采样后的对比图/'
img_b_RGB.save(root_path + '上采样_Bilinear2d_RGB.jpg')
img_b.save(root_path + '上采样_Bilinear2d.jpg')
img_l.save(root_path + '上采样_lingNearest2d.jpg')
im.save(root_path + '原图.jpg')

### 3. 按照性别划分子文件夹，分别保存男和女的人脸图片

In [58]:
import os
import random
import shutil
from shutil import copy2

In [67]:
# 获取保存好的上采样图像
data_path = '../face_data/sex_upsample/img/'
img_dirlist = os.listdir(data_path) # 获取该目录下图像名称列表

female_save_path = '../face_data/sex_upsample/Female_Male/female/'
male_save_path = '../face_data/sex_upsample/Female_Male/male/'
for img_name in img_dirlist:
    img_lable = img_name.split('.')[0]
    if img_lable == 'female':
        copy2(data_path + img_name, female_save_path) # 复制文件
    else:
        copy2(data_path + img_name, male_save_path)

### 4. 划分训练集、验证集、测试集

- 安照torch的数据保存要求保存好图像数据。
- 把图像按8：1：1保存到训练集、验证集、测试集。每个集中分别把同一类别的图像保存到同一文件夹中，这里共有两个文件夹，female和male。
- 这样标签也能很方便的保存下来。

In [79]:
# 调用util中的data_set_split函数来划分训练集、验证集、测试集
# from util import data_set_split
# src_data_folder = '../face_data/sex_upsample/Female_Male' # 源文件
# target_data_folder = '../face_data/sex/' # 目标文件
# data_set_split(src_data_folder, target_data_folder)

In [77]:
import os
import random
from shutil import copy2

def data_set_split(src_data_folder, target_data_folder, train_scale=0.8, val_scale=0.1, test_scale=0.1):

    '''
    读取源数据文件夹，生成划分好的文件夹，分为trian、valid、test三个文件夹进行
    :param src_data_folder: 源文件夹
    :param target_data_folder: 目标文件夹
    :param train_scale: 训练集比例
    :param val_scale: 验证集比例
    :param test_scale: 测试集比例
    :return:
    '''

    print("开始数据集划分")
    class_names = os.listdir(src_data_folder)
    # 在目标目录下创建文件夹
    split_names = ['train', 'valid', 'test']
    for split_name in split_names:
        split_path = os.path.join(target_data_folder, split_name)
        if os.path.isdir(split_path):
            pass
        else:
            os.mkdir(split_path)
        # 然后在split_path的目录下创建类别文件夹
        for class_name in class_names:
            class_split_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_split_path):
                pass
            else:
                os.mkdir(class_split_path)

    # 按照比例划分数据集，并进行数据图片的复制
    # 首先进行分类遍历
    for class_name in class_names:
        current_class_data_path = os.path.join(src_data_folder, class_name)
        current_all_data = os.listdir(current_class_data_path)
        current_data_length = len(current_all_data)
        current_data_index_list = list(range(current_data_length))
        random.shuffle(current_data_index_list)

        train_folder = os.path.join(os.path.join(target_data_folder, 'train'), class_name)
        val_folder = os.path.join(os.path.join(target_data_folder, 'valid'), class_name)
        test_folder = os.path.join(os.path.join(target_data_folder, 'test'), class_name)
        train_stop_flag = current_data_length * train_scale
        val_stop_flag = current_data_length * (train_scale + val_scale)
        current_idx = 0
        train_num = 0
        val_num = 0
        test_num = 0
        for i in current_data_index_list:
            src_img_path = os.path.join(current_class_data_path, current_all_data[i])
            if current_idx <= train_stop_flag:
                copy2(src_img_path, train_folder)
                # print("{}复制到了{}".format(src_img_path, train_folder))
                train_num = train_num + 1
            elif (current_idx > train_stop_flag) and (current_idx <= val_stop_flag):
                copy2(src_img_path, val_folder)
                # print("{}复制到了{}".format(src_img_path, val_folder))
                val_num = val_num + 1
            else:
                copy2(src_img_path, test_folder)
                # print("{}复制到了{}".format(src_img_path, test_folder))
                test_num = test_num + 1

            current_idx = current_idx + 1

        print("*********************************{}*************************************".format(class_name))
        print(
            "{}类按照{}：{}：{}的比例划分完成，一共{}张图片".format(class_name, train_scale, val_scale, test_scale, current_data_length))
        print("训练集{}：{}张".format(train_folder, train_num))
        print("验证集{}：{}张".format(val_folder, val_num))
        print("测试集{}：{}张".format(test_folder, test_num))

In [78]:
src_data_folder = '../face_data/sex_upsample/Female_Male' # 源文件
target_data_folder = '../face_data/sex/' # 目标文件
data_set_split(src_data_folder, target_data_folder)

开始数据集划分
*********************************female*************************************
female类按照0.8：0.1：0.1的比例划分完成，一共1566张图片
训练集../face_data/sex/train\female：1253张
验证集../face_data/sex/valid\female：157张
测试集../face_data/sex/test\female：156张
*********************************male*************************************
male类按照0.8：0.1：0.1的比例划分完成，一共2395张图片
训练集../face_data/sex/train\male：1917张
验证集../face_data/sex/valid\male：239张
测试集../face_data/sex/test\male：239张


- 数据预处理完成